# Relatório 2 - Regressao Multivariável

##  Estimar o preço de um imóvel a partir de suas características

### Aluno: Leandro Assis dos Santos - DRE 120032476

In [1]:
# Importação das bibliotecas utilziadas ao longo do projeto
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import LinearSVR
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import KFold
from scipy.stats import pearsonr
from statistics import mean

In [2]:
# Importação dos conjutos de amostras
dados_treino = pd.read_csv("conjunto_de_treinamento.csv", delimiter=",", decimal=".")
dados_teste = pd.read_csv("conjunto_de_teste.csv", delimiter=",", decimal=".")

# Análise dos dados

Primeiramente faz-se uma análise dos dados e de seus significados a fim de entende-los e idealizar possíveis ajustes e modelos que se adequariam ao mesmos. Para essa análise, fez-se a visualização dos conjuntos de treino e teste como tabela e os dos gráficos das variáveis em relação ao alvo.

## Análise dos conjuntos de dados

Através da análise das tabelas é possível notar que existem dados categóricos (Ex. colunas "bairro", "tipo" e etc) que deverão ser convertidos em valores numéricos futuramente. É notável também que a coluna "diferenciais" apresenta dados redundantes, uma vez que lista dados que já são apresentados nas colunas posteriores.

In [32]:
#dados_treino.head()

In [33]:
#dados_teste.head()

## Análise dos gráficos

In [5]:
# para ver os gráficos basta descomentar as linhas seguintes
#for coluna in dados_treino.columns:
#     dados_treino.plot.scatter(x=coluna, y='preco')

Para aprofundar o entendimento quanto às características do conjunto de dados gerou-se os gráficos de cada variável em relação com o alvo. Foi possível perceber alguns outliers nas colunas "area_util", "area_extra" e "vagas", já que imóveis cadastrados com quantidades muito acima da média possuiam preços ridiculamente baixos. Os outliers serão tratados futuramente no decorrer do projeto.

Através da análise dos gráficos é possível identificar a correlação de algumas variáveis com o alvo. Dentre elas destacam-se as colunas "quartos" e "suites" que, assim como esperado, apresentam uma correlação positiva forte com o preço final do imóvel.

In [6]:
colunas_com_outliers = ['area_util', 'area_extra', 'vagas']

# Tratamento dos dados

Após entender como os dados se comportam e identificar tratamentos e conversões que devem ser feitas no conjunto de dados, faz-se os seguintes processos para limpar os dados.

## Remoção de colunas e conversão de variáveis não numéricas

Nesta etapa remove-se as colunas "Id" e "diferenciais" que foram julgadas como irrelevantes para a predição do modelo. Além disso, troca-se a informação sobre os bairros da coluna "bairro" pela classificação arbitrária em relação ao preço médio das casas em determinado bairro. Determinado bairro pode ser classificado como "pobre", "mediano", "normal", "bom", "luxo", "5_estrelas".

In [7]:
# id - remover
# tipo, tipo_vendedor - get_dummies
# bairro - transformar em preço_medio do bairro > x vezes a media dos imoveis
# vagas - tirar outliers
# area_util - tirar outliers
# area_extra - tirar outliers
# diferenciais - remover 

dados_treino = dados_treino.drop(columns=['Id', 'diferenciais'])
dados_teste = dados_teste.drop(columns=['Id', 'diferenciais'])

In [8]:
media_imoveis = dados_treino['preco'].mean()

def classifica_bairros(df):
    classificacao_bairros = {}

    for bairro in df['bairro']:
        media_preco = dados_treino[dados_treino['bairro'] == bairro]['preco'].mean()

        if media_preco <= media_imoveis/6:
            classificacao_bairros[bairro] = 'pobre'
        elif media_preco <= media_imoveis/3:
            classificacao_bairros[bairro] = 'mediano'
        elif media_preco <= media_imoveis:
            classificacao_bairros[bairro] = 'normal'
        elif media_preco >= media_imoveis*3:
            classificacao_bairros[bairro] = '5_estrelas'
        elif media_preco >= media_imoveis*1.5:
            classificacao_bairros[bairro] = 'luxo'
        elif media_preco >= media_imoveis:
            classificacao_bairros[bairro] = 'bom'

    return classificacao_bairros

dados_teste['bairro'] = dados_teste['bairro'].map(classifica_bairros(dados_teste))
dados_treino['bairro'] = dados_treino['bairro'].map(classifica_bairros(dados_treino))

## Substituindo valores NaN

Após converter todos os dados da coluna bairro, verifica-se a existência de algum bairro no conjunto de teste que não existia no conjunto de treino. Essa tarefa é simplificada pois, no trecho de código acima, caso o bairro não fosse encontrado no conjunto de treino, seria preenchido como NaN.

In [9]:
print("Dados NULL por coluna no conjunto de treino")
print(dados_treino.isnull().sum(), end='\n\n')
print("Dados NULL por coluna no conjunto de teste")
print(dados_teste.isnull().sum())

Dados NULL por coluna no conjunto de treino
tipo              0
bairro            0
tipo_vendedor     0
quartos           0
suites            0
vagas             0
area_util         0
area_extra        0
churrasqueira     0
estacionamento    0
piscina           0
playground        0
quadra            0
s_festas          0
s_jogos           0
s_ginastica       0
sauna             0
vista_mar         0
preco             0
dtype: int64

Dados NULL por coluna no conjunto de teste
tipo              0
bairro            4
tipo_vendedor     0
quartos           0
suites            0
vagas             0
area_util         0
area_extra        0
churrasqueira     0
estacionamento    0
piscina           0
playground        0
quadra            0
s_festas          0
s_jogos           0
s_ginastica       0
sauna             0
vista_mar         0
dtype: int64


In [10]:
dados_teste[['bairro']] = dados_teste[['bairro']].fillna('normal')

Após executar a célula anterior, é garantido que todas as colunas de ambos os conjuntos de dados estão preenchidas.

## Splitando variáveis categóricas

Como foi percebido durante a análise de dados, é necessário converter as variáveis categóricas em variáveis booleanas. Para isso utiliza-se a função get_dummies abaixo.

In [11]:
valores_a_trocar = ['tipo', 'tipo_vendedor', 'bairro']

dados_treino = pd.get_dummies(dados_treino, columns=valores_a_trocar)
dados_teste = pd.get_dummies(dados_teste, columns=valores_a_trocar)

Existe um imóvel no conjunto de treino pertencente a um tipo que não existe no conjunto de teste, isso resulta em uma coluna a mais no conjunto de treino após o get_dummies. Para lidar com isso, cria-se a referida coluna no conjunto de teste.

In [12]:
dados_teste['tipo_Quitinete'] = [0 for indice in range(len(dados_teste))] # cria uma coluna em dados_teste que não existia
dados_teste['bairro_pobre'] = [0 for indice in range(len(dados_teste))] # cria uma coluna em dados_teste que não existia

## Removendo Outliers

Como discutido em sala de aula e por recomendação do professor, remove-se os outliers de preõ do conjunto de treino. Porém, como percebido durante a análise dos gráficos, existem outras variáveis que apresentam valores discrepantes. Por conta disso, resolvi passar um filtro em todas as colunas com outliers visíveis.

In [13]:
# remove as samples que possuem alvo > 10M ou alvo < 50K
dados_treino = dados_treino[(dados_treino['preco'] > 50000)&(dados_treino['preco'] < 10000000)]

In [14]:
# Converte todas as colunas para float a fim de poder fazer comparações aritméticas
dados_treino = dados_treino.astype(float)
dados_teste = dados_teste.astype(float)

In [15]:
# substitui os valores incompativeis com o valor médio de cada coluna por NaN
for coluna in colunas_com_outliers:
    Q1 = np.percentile(dados_treino[coluna], 25,
                   method = 'midpoint')
 
    Q3 = np.percentile(dados_treino[coluna], 75,
                   method = 'midpoint')
    IQR = Q3 - Q1
 
    max_ = Q3+1.5*IQR
    min_ = Q1-1.5*IQR
    
 
    dados_treino.loc[dados_treino[coluna] < min_, coluna] = np.nan
    dados_treino.loc[dados_treino[coluna] > max_, coluna] = np.nan

In [16]:
dados_treino.isnull().sum()

quartos                          0
suites                           0
vagas                          191
area_util                      245
area_extra                     549
churrasqueira                    0
estacionamento                   0
piscina                          0
playground                       0
quadra                           0
s_festas                         0
s_jogos                          0
s_ginastica                      0
sauna                            0
vista_mar                        0
preco                            0
tipo_Apartamento                 0
tipo_Casa                        0
tipo_Loft                        0
tipo_Quitinete                   0
tipo_vendedor_Imobiliaria        0
tipo_vendedor_Pessoa Fisica      0
bairro_5_estrelas                0
bairro_bom                       0
bairro_luxo                      0
bairro_mediano                   0
bairro_normal                    0
bairro_pobre                     0
dtype: int64

In [17]:
# preenche os dados NaN gerados pela remoção dos outliers com a média da coluna
for coluna in colunas_com_outliers:
    dados_treino[[coluna]] = dados_treino[[coluna]].fillna(dados_treino[coluna].median())

In [34]:
# dados limpos pós filtragem de outliers
#dados_treino

## Análise da correlação das variáveis

Neste ponto todas as variáveis já estão em valores numéricos e é possível verificar numericamente a correlação de cada uma com o alvo. Nas células seguintes, calcula-se e apresenta-se de forma ordenada o módulo do coeficiente de Pearson para cada variável.

In [19]:
# calcula o coeficiente de Pearson para cada coluna em relação ao alvo
pearson_coef = {}

for coluna in dados_treino.columns:
    coef = round(abs(pearsonr(dados_treino[coluna], dados_treino['preco'])[0]),4)
    if type(coef) not in [float, int] or coluna == 'preco':
        pearson_coef[coluna] = coef
        
from operator import itemgetter

coef_ordenados = sorted(pearson_coef.items(), key=itemgetter(1))

coef_ordenados = [[tupla[0], tupla[1]] for tupla in coef_ordenados]

coef_ordenados = sorted(coef_ordenados, key=itemgetter(1))
coef_ordenados

C:\Users\assis\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\stats\_stats_py.py:4068: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


[['quadra', 0.0027],
 ['s_ginastica', 0.003],
 ['bairro_pobre', 0.0133],
 ['s_jogos', 0.0137],
 ['tipo_Loft', 0.0153],
 ['tipo_Quitinete', 0.0182],
 ['tipo_vendedor_Imobiliaria', 0.0297],
 ['tipo_vendedor_Pessoa Fisica', 0.0297],
 ['estacionamento', 0.0452],
 ['churrasqueira', 0.046],
 ['bairro_normal', 0.0469],
 ['playground', 0.0538],
 ['bairro_luxo', 0.0609],
 ['s_festas', 0.0695],
 ['piscina', 0.0778],
 ['tipo_Apartamento', 0.1032],
 ['tipo_Casa', 0.1086],
 ['bairro_5_estrelas', 0.1194],
 ['sauna', 0.1377],
 ['bairro_mediano', 0.1449],
 ['bairro_bom', 0.1685],
 ['vista_mar', 0.1919],
 ['vagas', 0.3792],
 ['area_util', 0.4852],
 ['quartos', 0.564],
 ['suites', 0.6867],
 ['preco', 1.0]]

Com os coeficientes numéricos listados acima, pode-se escolher manualmente as variáveis com maior correlação para compor  o modelo preditivo. Entretanto para diferenciar do realizado no Trabalho 1 e, dado a pequena quantidade de variáveis (e a presença de coeficientes muito mais relevantes na maioria em comparação ao Trabalho anterior), decidiu-se deixar a decisão da escolha de variáveis para um dos métodos de feature selection implementados na biblioteca sklearn.

In [20]:
indice_aceitavel = 0 # todas as variaveis que possuem Pearson não nulo
variaveis_escolhidas = []

for item in coef_ordenados[indice_aceitavel:]:
    variaveis_escolhidas.append(item[0])

dados_treino = dados_treino[variaveis_escolhidas]
dados_teste = dados_teste[variaveis_escolhidas[:-1]]

## Separação dos dados

In [21]:
# separacao do conjunto de treino em alvo e features
X = dados_treino.drop(columns=['preco'])
Y = np.array(dados_treino['preco'])

## Escalando os dados

In [22]:
# escalando os dados
standard_scaler =  StandardScaler()

X = standard_scaler.fit_transform(X)
dados_teste = standard_scaler.transform(dados_teste)

# Escolhendo variáveis e criando os modelos preditivos

Dada a natureza do problema resolvi utilizar dois modelos combinados, KNN e Random Forest. Ambos os modelos foram calibrados e metrificados incontáveis vezes até obter uma configuração de hiperparâmetros satisfatória.

A avaliação dos modelos é feita através de validação cruzada (utilizando o módulo KFold), para cada validação é computado o RMSPE da configuração de modelo naquele determinado subconjunto de dados. Ao final da última validação, é computada a média do RMSPE da configuração. A configuração com menor RMSPE médio foi escolhida para compor o modelo final.

Para escolher as variáveis foi utilizado a função SequentialFeatureSelector que computa um algoritmo greedy a fim de obter a melhor combinação de features do conjunto de dados. O ajuste do "n_features_to_select" foi feito de forma empírica avaliando a variação no RMSPE médio de cada configuração do modelo.

In [23]:
kf = KFold(n_splits=3, shuffle=True, random_state=1456) # cria instancia utilizada para CV no teste dos modelos

def rmspe(y_true, y_pred): # função para calcular o RMSPE 
    return np.sqrt(np.mean(np.square(((y_true - y_pred) / y_true))))

## Criando KNN

In [24]:
print("  K   |    RMSPE   ")
for numero_kneighbors in range(1, 9, 2):
    regressorKNN = KNeighborsRegressor(n_neighbors=numero_kneighbors, weights='uniform')
    
    sfs = SequentialFeatureSelector(regressorKNN, n_features_to_select=11, direction='backward')
    sfs.fit(X, Y)
    
    erro_percentual_medio = []
    for indice_treino, indice_teste in kf.split(X):
        regressorKNN.fit(sfs.transform(X[indice_treino]), Y[indice_treino])

        predicao = regressorKNN.predict(sfs.transform(X[indice_teste]))
        erro_percentual_medio.append(rmspe(predicao, Y[indice_teste]))
    
    print(  "%d    " %numero_kneighbors, end='')
    print(f"    {round(mean(erro_percentual_medio), 4)}")

  K   |    RMSPE   
1        0.5106
3        0.3917
5        0.3646
7        0.3678


## Criando Random Forest

In [25]:
print("  N   |   RMSPE  ")
for numero in range(33, 45, 3):
    regressorRF = RandomForestRegressor(n_estimators=numero)
     
    sfs = SequentialFeatureSelector(regressorRF, n_features_to_select=10, direction='backward')
    sfs.fit(X, Y)
    
    erro_percentual_medio = []
    for indice_treino, indice_teste in kf.split(X):
        regressorRF.fit(sfs.transform(X[indice_treino]), Y[indice_treino])

        predicao = regressorRF.predict(sfs.transform(X[indice_teste]))
        erro_percentual_medio.append(rmspe(predicao, Y[indice_teste]))
    
    print("%d    " %numero, end='')
    print(f"  {round(mean(erro_percentual_medio), 4)}")

  N   |   RMSPE  
33      0.3623
36      0.3488
39      0.3531
42      0.349


## Criando modelo final

O modelo final utiliza das predições dos modelos de KNN e Random Forest para calcular suas predições. Basicamente o modelo final calcula a média ponderada entre as n-ésimas predições do KNN e do Random Forest utilizando como pesos o inverso do RMSPE médio calculado durante o fitting do modelo final.

O RMSPE médio calculado durante o fitting é a média dos RMSPE calculados em cada validação cruzada.

In [26]:
class Regressor():
    def __init__(self):
        self.RFC = RandomForestRegressor(n_estimators=37)
        self.KNN = KNeighborsRegressor(n_neighbors=6, weights='uniform')
        self.modelos = [self.KNN, self.RFC]
    def fit(self, X, Y):
        self.rmspe_modelos = []
        self.sfs = [SequentialFeatureSelector(self.modelos[0], n_features_to_select=11, direction='backward'), \
                    SequentialFeatureSelector(self.modelos[1], n_features_to_select=10, direction='backward')]
        for indice, modelo in enumerate(self.modelos): 
            self.sfs[indice].fit(X, Y)
            self.modelos[indice] = modelo.fit(self.sfs[indice].transform(X), Y)
            
            erro_percentual_medio = []
            for indice_treino, indice_teste in kf.split(X):
                modelo.fit(self.sfs[indice].transform(X[indice_treino]), Y[indice_treino])

                predicao_teste = modelo.predict(self.sfs[indice].transform(X[indice_teste]))
                predicao_treino = modelo.predict(self.sfs[indice].transform(X[indice_treino]))
                erro_percentual_medio.append(rmspe(predicao_teste, Y[indice_teste]))
            self.rmspe_modelos.append(mean(erro_percentual_medio))
    def predict(self, X_alvo):
        predicao_modelos = [modelo.predict(self.sfs[indice].transform(X_alvo)) for indice, modelo in enumerate(self.modelos)]
        
        peso_KNN = 1/self.rmspe_modelos[0]
        peso_RFC = 1/self.rmspe_modelos[1]
        
        return [ (predicao_modelos[0][indice]*peso_KNN + predicao_modelos[1][indice]*peso_RFC)/(peso_RFC+peso_KNN) \
                for indice in range(len(X_alvo))]
        
            

In [27]:
regressor = Regressor()

rmspe_teste = []
rmspe_treino = []

## Avaliação do modelo

Para avaliar o modelo computa-se o RMSPE das previsões para a amostra de treino e teste. Ao final computa-se as médias dos RMSPE para ambas as amostras.

In [28]:
# faz a CV do modelo final
print("  TESTE  |  TREINO")
for indice_treino, indice_teste in kf.split(X):
    regressor.fit(X[indice_treino], Y[indice_treino])
    yteste = regressor.predict(X[indice_teste])
    ytreino = regressor.predict(X[indice_treino])
    
    score_teste = round(rmspe(yteste, Y[indice_teste]), 4)
    score_treino = round(rmspe(ytreino, Y[indice_treino]) ,4)
    
    rmspe_teste.append(score_teste)
    rmspe_treino.append(score_treino)
    
    print(f"   {score_teste}     ", end='')
    print(score_treino)

print("-------------------")
print("  TESTE  |  TREINO")
print(f"   {round(mean(rmspe_teste), 4)}     ", end='')
print(round(mean(rmspe_treino), 4))

  TESTE  |  TREINO
   0.3866     0.2263
   0.2842     0.2533
   0.4252     0.2594
-------------------
  TESTE  |  TREINO
   0.3653     0.2463


# Geração do arquivo de respostas

In [35]:
regressor.fit(X, Y)

x_resposta = dados_teste

y_resposta = regressor.predict(x_resposta)

In [36]:
id_solicitante = [x for x in range(2000)]

dados_resposta = pd.DataFrame(list(zip(id_solicitante, y_resposta)), columns=['Id', 'preco'])

In [37]:
dados_resposta.to_csv("arquivo_resposta.csv", index=False)